In [1]:
import stream_time_reversability as STR
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import tstrippy
import datetime
import multiprocessing as mp
from astropy import units as u
from astropy.coordinates import SkyCoord
import numericalErrorFunctions as NEF

In [2]:
integrationtime = .5 
currenttime = 0 

In [3]:
MWparams = tstrippy.Parsers.pouliasis2017pii()
dtfactor = 1/20
staticgalaxy = ["pouliasis2017pii", MWparams]
G = MWparams[0]
M = 1e5
a = 5e-3
NP = int(1e3)
aplum = tstrippy.ergodic.convertHalfMassRadiusToPlummerRadius(a)
tau = STR.plummer_dynamical_time([G,M,a])

In [4]:
targetGC = "NGC5139"
initialkinematics = STR.pick_GC_get_kinematics(targetGC)

In [5]:
integrationparameters = STR.prepare_integration_arguments(currenttime=currenttime,
                                                          integrationtime=integrationtime,
                                                          tdyn=tau,
                                                          alpha=dtfactor)

In [6]:
args = initialkinematics, staticgalaxy, integrationparameters
orbit, timestamps, comptime = STR.integrate_host_orbit_back(args)

integration time:  0:00:00.092845


In [ ]:
inithostperturber = [timestamps, *orbit, G, M, aplum ]
initialkinematics_stream = tstrippy.ergodic.isotropicplummer(G,M,a,NP)
integrationparameters_stream = [timestamps[0],*integrationparameters[1:]]